In [46]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib
#%matplotlib notebook
%matplotlib qt5
#%matplotlib inline
from matplotlib import pyplot as plt

import seaborn as sns

matplotlib.rcParams['figure.figsize'] = (20.0, 10.0) # bigger figure!


sns.set() # better looking figs


In [2]:
# our own set of small helper functions for plotting, etc
from utils import plot_embedding, plot_compare_embeddings, show_heatmap, plot_confusion_matrix

In [3]:
df = pd.read_csv("../fulldata.csv")
df["clipId"] = df["clipName"].apply(lambda x: x[-8:-6])

# re-order columns + keep only useful ones
df = df[['pptID',
 'fileName',
 'condition',
 'age',
 'gender',
 'nationality',
 'firstLang',
 'trial',
 'clipId',
 'freetext',
 'q01',
 'q02',
 'q03',
 'q04',
 'q05',
 'q06',
 'q07',
 'q08',
 'q09',
 'q10',
 'q11',
 'q12',
 'q13',
 'q14',
 'q15',
 'q16',
 'q17',
 'q18',
 'q19',
 'q20',
 'q21',
 'q22',
 'q23',
 'q24',
 'q25',
 'q26',
 'q27',
 'q28',
 'q29',
 'q30']]

df

,pptID,fileName,condition,age,gender,nationality,firstLang,trial,clipId,freetext,...,q21,q22,q23,q24,q25,q26,q27,q28,q29,q30
0,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,1,04,PLAY TOGETHER THE CARD GAME TO ENJOY THEM . IN...,...,2,2,1,2,1,2,2,2,2,2
1,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,2,09,PLAYING TOGETHER FOR TWO CHILDREN . TO PLAY CA...,...,3,4,3,3,3,4,3,4,3,3
2,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,3,16,PLAYING TOGETHER THE GAME. PLAY TO LEARNING TH...,...,3,4,3,3,3,3,3,4,3,3
3,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,4,02,PLAYING TO LEARNING TOGETHER. TO PLAY INTEREST...,...,3,4,3,3,2,2,3,3,3,3
4,94,10kinos8b34va6_data.csv,1,23,Male,American,English,1,15,I notice that they slow down towards the end m...,...,1,1,2,1,1,2,2,2,2,2
5,94,10kinos8b34va6_data.csv,1,23,Male,American,English,2,02,The child on the right seems to be taking over...,...,1,0,1,1,1,3,4,1,3,0
6,94,10kinos8b34va6_data.csv,1,23,Male,American,English,3,08,"Both seem pretty calm, but the child on the le...",...,4,1,4,1,2,1,1,2,3,2
7,94,10kinos8b34va6_data.csv,1,23,Male,American,English,4,11,These children seem to be working together pre...,...,1,1,1,1,1,1,2,2,1,1
8,155,10kinqv5zq7rl5_data.csv,2,28,Male,American,English,1,12,They got along quite well and helped each othe...,...,0,4,3,2,2,4,2,2,4,2
9,155,10kinqv5zq7rl5_data.csv,2,28,Male,American,English,2,07,They played separately largely. They were fine...,...,3,0,4,1,3,4,3,3,0,2


Rename `qXX` columns with the names of the actual constructs

In [4]:
constructs=["Sad", "Happy", "Angry", "Excited", "Calm", "Friendly", "Aggressive", "Engaged", "Distracted", "Bored", "Frustrated","Dominant","Submissive"]

index = df.columns.tolist()
index = index[0:10] + ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + [c for c1 in constructs for c in ['left' + c1, 'right' + c1]]
df.columns=index
df

,pptID,fileName,condition,age,gender,nationality,firstLang,trial,clipId,freetext,...,leftDistracted,rightDistracted,leftBored,rightBored,leftFrustrated,rightFrustrated,leftDominant,rightDominant,leftSubmissive,rightSubmissive
0,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,1,04,PLAY TOGETHER THE CARD GAME TO ENJOY THEM . IN...,...,2,2,1,2,1,2,2,2,2,2
1,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,2,09,PLAYING TOGETHER FOR TWO CHILDREN . TO PLAY CA...,...,3,4,3,3,3,4,3,4,3,3
2,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,3,16,PLAYING TOGETHER THE GAME. PLAY TO LEARNING TH...,...,3,4,3,3,3,3,3,4,3,3
3,186,10kin7dm75u231_data.csv,2,30,Female,Indian,Tamil,4,02,PLAYING TO LEARNING TOGETHER. TO PLAY INTEREST...,...,3,4,3,3,2,2,3,3,3,3
4,94,10kinos8b34va6_data.csv,1,23,Male,American,English,1,15,I notice that they slow down towards the end m...,...,1,1,2,1,1,2,2,2,2,2
5,94,10kinos8b34va6_data.csv,1,23,Male,American,English,2,02,The child on the right seems to be taking over...,...,1,0,1,1,1,3,4,1,3,0
6,94,10kinos8b34va6_data.csv,1,23,Male,American,English,3,08,"Both seem pretty calm, but the child on the le...",...,4,1,4,1,2,1,1,2,3,2
7,94,10kinos8b34va6_data.csv,1,23,Male,American,English,4,11,These children seem to be working together pre...,...,1,1,1,1,1,1,2,2,1,1
8,155,10kinqv5zq7rl5_data.csv,2,28,Male,American,English,1,12,They got along quite well and helped each othe...,...,0,4,3,2,2,4,2,2,4,2
9,155,10kinqv5zq7rl5_data.csv,2,28,Male,American,English,2,07,They played separately largely. They were fine...,...,3,0,4,1,3,4,3,3,0,2


Compute, for each left/right constructs, the differences and sums. This gives us an insight on the inbalance of the given construct between the children, and the overall 'magnitude' of the construct in the clip.

In [5]:
for c in constructs:
    df["diff"+c] = abs(df["left" + c] - df["right" + c])
    df["sum"+c] = df["left" + c] + df["right" + c] - 4
    
# create 2 lists of columns names, one for left/right constructs, one for diff/sum constructs
columnsLeftRight=[]
columnsDiffSum=[]

for c in constructs:
    columnsLeftRight.append("left" + c)
    columnsLeftRight.append("right" + c)
    
    columnsDiffSum.append("diff" + c)
    columnsDiffSum.append("sum" + c)
    

df[df["condition"]==2].to_csv("data_fullscene.csv")
df[df["condition"]==1].to_csv("data_skel.csv")

# work with differences & sum for each constructs
selectedColumns=columnsDiffSum

## work with left child/right child for each constructs
#selectedColumns=columnsLeftRight

allQuestionsDiffSum = ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + columnsDiffSum
allQuestionsLeftRight = ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + columnsLeftRight
allQuestions = ["Competing", "Cooperating", "PlaySeparate", "PlayTogether"] + selectedColumns


*we define here several useful partial views of the main dataframe*

In [6]:
fullscene_df=df[df["condition"]==2] # full scene

# the responses to the 26 left/right Likert-scale questions
fullscene_ratings_df=fullscene_df[selectedColumns].astype(float)
fullscene=fullscene_ratings_df.values # the underlying numpy array

# clip names
fullscene_labels=fullscene_df["clipId"].values

# mean ratings per clip
fullscene_means=fullscene_df.groupby(["clipId"]).mean()[selectedColumns]


skel_df=df[df["condition"]==1] # skeleton

# the responses to the 26 left/right Likert-scale questions
skel_ratings_df=skel_df[selectedColumns].astype(float)
skel=skel_ratings_df.values # the underlying numpy array

# clip names
skel_labels=skel_df["clipId"].values

# mean ratings per clip
skel_means=skel_df.groupby(["clipId"]).mean()[selectedColumns]

In [7]:
meanvar_full_ratings=fullscene_df.groupby(["clipId"]).std()[columnsLeftRight].T.mean()
meanvar_skel_ratings=skel_df.groupby(["clipId"]).std()[columnsLeftRight].T.mean()

pd.DataFrame([meanvar_full_ratings,meanvar_skel_ratings],index=["mean stddev fullscene ratings","mean stddev skel ratings"]).T

,mean stddev fullscene ratings,mean stddev skel ratings
clipId,,
01,0.913077,1.068177
02,0.956018,0.984399
03,1.069666,0.916187
04,1.035666,1.056938
05,1.016512,0.936117
06,1.009450,0.939598
07,1.087390,1.037589
08,1.046094,0.956287
09,1.044294,1.018345


In [8]:
import krippendorff

krip={}

for clipName, group in fullscene_df[["clipId"] + allQuestionsLeftRight].groupby(["clipId"]):
    krip[clipName]=(krippendorff.alpha(group.values[:,1:].astype(int),level_of_measurement='interval'), group.shape[0])

for clipName, group in skel_df[["clipId"] + allQuestionsLeftRight].groupby("clipId"):
    krip[clipName]=krip[clipName] + (krippendorff.alpha(group.values[:,1:].astype(int),level_of_measurement='interval'), group.shape[0])

    
krippendorff_df=pd.DataFrame.from_dict(krip,orient="index", columns=["alpha(fullscene)", "N(fullscene)", "alpha(skel)", "N(skel)"])

show_heatmap(krippendorff_df[["alpha(fullscene)", "alpha(skel)"]], cmap="summer")




,alpha(fullscene),alpha(skel)
01,0.446443,0.186386
02,0.18072,0.269889
03,0.392881,0.368891
04,0.44438,0.26173
05,0.327889,0.283435
06,0.46281,0.359461
07,0.0914201,0.235908
08,0.339198,0.311564
09,0.0974896,0.0578229
10,0.396426,0.0861424


Comparing the agreement scores in the fullscene vs skeleton-only videos, using a T-test:

In [9]:
from scipy.stats import ttest_rel

fullscene_krip = krippendorff_df["alpha(fullscene)"]
skel_krip = krippendorff_df["alpha(skel)"]

ttest_rel(fullscene_krip, skel_krip)

Ttest_relResult(statistic=2.955422785203005, pvalue=0.008124095387554918)

# Latent constructs

The first step of the analysis looks at latent constructs.

Our initial data contains responses to 22 questions (ie, 22 degrees of freedom). The question is: can those 22 DoFs be grouped into a smaller number of *latent* constructs that would effectively encapsulate the differences observed in the reponses between video clips.

Three approaches are explored:
- Principal component analysis (PCA)
- Principal component analysis (purely for dimensionality reduction) followed by a linear discriminant analysis (LDA) that aims at maximising inter-class distances (ie, inter-clips ratings) while minimizing intra-class distances (ie, the differences between ratings for a given clip).
- Explorative Factor Analysis (EFA)

## PCA

In [10]:
from sklearn.decomposition import PCA

We compute the PCA transformation with the responses to the *fullscene* stimuli.

We then project both the *fullscene* and the *skeletal-only* responses in this PCA space, effectively reducing the dimensionality of our data from 22 to `nb_components` (ie, 6).

In [11]:
nb_components = 6

fullscene_pca_model=PCA(n_components=nb_components).fit(fullscene)

fullscene_pca = fullscene_pca_model.transform(fullscene)
fullscene_means_pca = fullscene_pca_model.transform(fullscene_means.values)

skel_pca_model = fullscene_pca_model

skel_pca = skel_pca_model.transform(skel)
skel_means_pca = skel_pca_model.transform(skel_means.values)

With 6 components, about 70% of the variance in the *fullscene* dataset is explained.

In [12]:
pd.DataFrame(fullscene_pca_model.explained_variance_ratio_).plot(ylim=[0,1])
print("Cumulative explained variance: %s" % fullscene_pca_model.explained_variance_ratio_.cumsum())

Cumulative explained variance: [0.32430671 0.44723373 0.54745441 0.61906194 0.67073193 0.70727606]


### Plotting of the embeding

`plot_embedding` plots each questionnaire's response when projected along the first 2 eigenvectors. Responses' colours correspond to the clips.

In [13]:
plot_embedding(fullscene_pca, fullscene_labels, fullscene_means_pca, fullscene_means.index, title="'Fullscene' dataset projection onto its PCA space", three_d=False)
plot_embedding(skel_pca, skel_labels, skel_means_pca, skel_means.index, title="'Skeletal' dataset projection onto its PCA space", three_d=False) 

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

For each clip, we can then plot the distance between its embedding based on fullscene ratings vs its embedding based on skeletal data only.

In [14]:
plot_compare_embeddings(skel_means_pca, fullscene_means_pca, skel_means.index, title="O: skeletal, X: fullscene", three_d=False)
plot_compare_embeddings(skel_means_pca, fullscene_means_pca, skel_means.index, title="O: skeletal, X: fullscene", three_d=True)

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

Computing the actual distance between clips in the two conditions shows that they are generally quite far apart. A straightforward PCA embedding does not seem to be effective to evidence similarities between our 2 conditions.

In [15]:
distances_pca=pd.DataFrame(np.power(np.sum(np.power(skel_means_pca - fullscene_means_pca, 2), axis=1), 0.5), index=skel_means.index, columns=["distance_pca"])
show_heatmap(distances_pca, cmap="summer_r")

,distance_pca
clipId,
01,4.65382
02,0.883654
03,1.73115
04,4.40108
05,1.2188
06,2.3967
07,2.65566
08,1.57035
09,1.54301


To answer the question: *does a PCA evidence common latent factors between our 2 conditions?*, we compute a PCA model *based on the skeletal data*, and compare the resulting PCA components with the ones found with the fullscene data.

We observe that the resulting loadings look very different.

In [16]:
# plot of PCA components, with fullscene vs skeleton components side-by-side


skel_pca_model=PCA(n_components=nb_components).fit(skel)

skel_pca = skel_pca_model.transform(skel)
skel_means_pca = skel_pca_model.transform(skel_means.values)



skel_pca_components = pd.DataFrame(skel_pca_model.components_,columns=columnsDiffSum).T
fullscene_pca_components = pd.DataFrame(fullscene_pca_model.components_,columns=columnsDiffSum).T

# merge PCA components into one dataframe, skel and fullscene side-by-side
pca_components=pd.concat([skel_pca_components, fullscene_pca_components], keys=["skel", "fullscene"], axis=1)
pca_components=pca_components.swaplevel(0,1,1).sort_index(1)


show_heatmap(pca_components[abs(pca_components)>0.2], m=-0.6, M=0.6)


/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


## LDA

We perform the LDA *on top of the PCA* as LDA typically requires O > 3 F, with O the nb of observations and F the nb of features (here, we have ~26 observations for originally 22 questions). Using the PCA as a dimensionality reduction tool, we bring down the number of degrees of freedom to 6.

In [17]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_nb_components = 4

fullscene_lda_model = LinearDiscriminantAnalysis(n_components=lda_nb_components, solver='svd')
fullscene_lda_model.fit(fullscene_pca, fullscene_labels)

fullscene_lda = fullscene_lda_model.transform(fullscene_pca_model.transform(fullscene))
fullscene_means_lda = fullscene_lda_model.transform(fullscene_pca_model.transform(fullscene_means.values))

skel_lda = fullscene_lda_model.transform(fullscene_pca_model.transform(skel))
skel_means_lda = fullscene_lda_model.transform(fullscene_pca_model.transform(skel_means.values))


/usr/lib/python3/dist-packages/sklearn/discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


Attention: the variance explained by the LDA transformation is the variance in the *PCA* space, not in the original 22-D space of the questionnaire!

In [18]:
print("Cumulative explained variance by LDA: %s" % fullscene_lda_model.explained_variance_ratio_.cumsum())

Cumulative explained variance by LDA: [0.44107238 0.70979231 0.8487203  0.92541401]


When projected in the PDA space, the clips in condition *skeleton* vs *fullscene* are much closer to one another.

In [19]:
distances_lda=pd.DataFrame(np.power(np.sum(np.power(skel_means_lda - fullscene_means_lda, 2), axis=1), 0.5), index=skel_means.index, columns=["distance_lda"])

distances = pd.concat([distances_pca, distances_lda], axis=1)
show_heatmap(distances, cmap="summer_r")


,distance_pca,distance_lda
clipId,,
01,4.65382,1.78732
02,0.883654,0.350891
03,1.73115,1.03464
04,4.40108,2.42992
05,1.2188,0.486269
06,2.3967,0.679606
07,2.65566,1.27701
08,1.57035,0.72836
09,1.54301,0.63264


In [20]:
plot_embedding(fullscene_lda, fullscene_labels,fullscene_means_lda, fullscene_means.index, title="LDA embedding, fullscene data", three_d=False)
plot_embedding(skel_lda, skel_labels,skel_means_lda, skel_means.index, title="LDA embedding, skeletal data", three_d=False)

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

In [21]:
plot_compare_embeddings(skel_means_lda, fullscene_means_lda, fullscene_means.index,three_d=False)
plot_compare_embeddings(skel_means_lda, fullscene_means_lda, skel_means.index,three_d=True)

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

To answer the question: *does a PCA evidence common latent factors between our 2 conditions?*, we can again create a new LDA model for the skeletal data, and print out the LDA components for *fullscene* vs *skeleton* side-by-side.

We observe that they still look very different.

In [22]:
# compute as well a LDA model from the skeletal PCA to compare components with fullscene
skel_lda_model = LinearDiscriminantAnalysis(n_components=lda_nb_components, solver='svd')
skel_lda_model.fit(skel_pca, skel_labels)


fullscene_lda_components = pd.DataFrame(fullscene_lda_model.scalings_[:,:lda_nb_components].T).T
skel_lda_components = pd.DataFrame(skel_lda_model.scalings_[:,:lda_nb_components].T).T

# merge PCA components into one dataframe, skel and fullscene side-by-side
lda_components=pd.concat([fullscene_lda_components, skel_lda_components], keys=["fullscene", "skel"], axis=1)
lda_components=lda_components.swaplevel(0,1,1).sort_index(1)

show_heatmap(lda_components[abs(lda_components)>0.2], m=-0.6, M=0.6)

/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


By multiplying the LDA loadings matrix with the PCA loadings matrix, we can compute the LDA loadings in terms of the original questions asked to the participants.

In [23]:
lda_fullscene_loadings=pd.DataFrame(np.dot(fullscene_pca_components, fullscene_lda_components), index=pca_components.index, columns=["LDA component %d" % (i+1) for i in range(lda_nb_components)])
lda_skel_loadings=pd.DataFrame(np.dot(skel_pca_components, skel_lda_components), index=pca_components.index, columns=["LDA component %d" % (i+1) for i in range(lda_nb_components)])
# merge loadings into one dataframe, skel and fullscene side-by-side
lda_loadings=pd.concat([lda_fullscene_loadings, lda_skel_loadings], keys=["fullscene","skel"], axis=1)
lda_loadings=lda_loadings.swaplevel(0,1,1).sort_index(1)

from scipy.stats import pearsonr

print("Pearson correlation between LDA components 'fullscene' vs 'skeletal'")
for i in range(1, lda_nb_components + 1):
    r, p=pearsonr(lda_loadings["LDA component %d" % i]["fullscene"].values, lda_loadings["LDA component %d" % i]["skel"].values)
    print("LDA component %d: r=%f, p=%f" % (i,r,p)) 


show_heatmap(lda_loadings[abs(lda_loadings)>=0.1], m=-0.6, M=0.6)

/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Pearson correlation between LDA components 'fullscene' vs 'skeletal'
LDA component 1: r=0.963956, p=0.000000
LDA component 2: r=-0.336108, p=0.093196
LDA component 3: r=-0.552121, p=0.003451
LDA component 4: r=-0.316252, p=0.115494


## Explorative Factor Analysis

The Python [factor_analyzer](https://factor-analyzer.readthedocs.io) module is a port of EFA from the R' `psych` package.

In [24]:
import factor_analyzer

rotation = 'promax'

nb_factors=3

fa_fullscene = factor_analyzer.FactorAnalyzer()
fa_fullscene.analyze(fullscene_ratings_df, nb_factors, rotation=rotation)
fullscene_loadings=fa_fullscene.loadings

fa_skel = factor_analyzer.FactorAnalyzer()
fa_skel.analyze(skel_ratings_df, nb_factors, rotation=rotation)
skel_loadings=fa_skel.loadings

Comparing the loadings for the *fullscene* vs the *skeletal* only data show that the first two factors are highly correlated. **This shows that, using factor analysis, we have uncovered latent constructs that are used by participants to describe the clips in both *fullscene* and *skeletal-only* conditions**.

In [77]:
# merge loadings into one dataframe, skel and fullscene side-by-side
loadings=pd.concat([fullscene_loadings, skel_loadings], keys=["fullscene","skel"], axis=1)
loadings=loadings.swaplevel(0,1,1).sort_index(1)

from scipy.stats import pearsonr

print("Pearson correlation between factors 'fullscene' vs 'skeletal'")
for i in range(1, nb_factors+1):
    r, p=pearsonr(loadings["Factor%d" % i]["fullscene"].values, loadings["Factor%d" % i]["skel"].values)
    print("Factor %d: r=%f, p=%f" % (i,r,p)) 
    
    
show_heatmap(loadings[abs(loadings)>=0.35])


Pearson correlation between factors 'fullscene' vs 'skeletal'
Factor 1: r=0.937367, p=0.000000
Factor 2: r=0.835589, p=0.000000
Factor 3: r=0.809878, p=0.000001


/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [81]:
print(loadings[abs(loadings)>=0.35].round(2).to_latex(bold_rows=True))

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{2}{l}{Factor1} & \multicolumn{2}{l}{Factor2} & \multicolumn{2}{l}{Factor3} \\
{} & fullscene &  skel & fullscene &  skel & fullscene &  skel \\
\midrule
\textbf{diffSad       } &      0.41 &  0.52 &       NaN &   NaN &       NaN &   NaN \\
\textbf{sumSad        } &       NaN &   NaN &      0.72 &  0.53 &       NaN &  0.49 \\
\textbf{diffHappy     } &      0.49 &  0.53 &       NaN &   NaN &       NaN &   NaN \\
\textbf{sumHappy      } &       NaN &   NaN &       NaN & -0.51 &     -0.55 &   NaN \\
\textbf{diffAngry     } &      0.40 &  0.62 &       NaN &   NaN &       NaN &   NaN \\
\textbf{sumAngry      } &       NaN &   NaN &      0.81 &  0.85 &       NaN &   NaN \\
\textbf{diffExcited   } &      0.53 &  0.63 &       NaN &   NaN &       NaN &   NaN \\
\textbf{sumExcited    } &       NaN &   NaN &       NaN &   NaN &     -0.71 &   NaN \\
\textbf{diffCalm      } &      0.45 &  0.63 &       NaN &   NaN &       NaN &   NaN \\
\textbf{sumC

In [ ]:
constructs=["Dominant", "Cooperative", "Competitive", "Friendly", "Aggressive", "Engaged", "Fearful", "Sad", "Content", "Angry", "Amused"]

loadings2 = pd.DataFrame()

for c in constructs:
    loadings2[c + ": diff."] = abs(loadings.T["left" + c] - loadings.T["right" + c])
    loadings2[c + ": mag."] = np.amax([abs(loadings.T["left" + c]), abs(loadings.T["right" + c])], axis=0)

show_heatmap(loadings2.T, cmap="summer_r")


In [26]:
fa_skel.get_factor_variance()


,Factor1,Factor2,Factor3
SS Loadings,4.805225,3.677328,3.590071
Proportion Var,0.184816,0.141436,0.138080
Cumulative Var,0.184816,0.326252,0.464332


In [27]:
fa_fullscene.get_factor_variance()

,Factor1,Factor2,Factor3
SS Loadings,3.961499,3.854104,3.598299
Proportion Var,0.152365,0.148235,0.138396
Cumulative Var,0.152365,0.300600,0.438996


### EFA embeddings

We can use the EFA space as a 'better' space to represent our clips, where the latent, composite constructs correspond to the main axis:

In [28]:
nb_of_factors=3
fullscene_efa = np.dot(fullscene,fullscene_loadings.values[:,:nb_of_factors])
fullscene_means_efa = np.dot(fullscene_means,fullscene_loadings.values[:,:nb_of_factors])
skel_efa = np.dot(skel,fullscene_loadings.values[:,:nb_of_factors])
skel_means_efa = np.dot(skel_means,fullscene_loadings.values[:,:nb_of_factors])

skel_pure_efa = np.dot(skel,skel_loadings.values[:,:nb_of_factors])
skel_pure_means_efa = np.dot(skel_means,skel_loadings.values[:,:nb_of_factors])


plot_embedding(fullscene_efa, fullscene_labels,fullscene_means_efa, fullscene_means.index, title="EFA-space embedding of the fullscene data", three_d=True)
plot_embedding(skel_efa, skel_labels,skel_means_efa, skel_means.index, title="EFA-space embedding of the skeletal data (EFA on fullscene data)", three_d=False)
plot_embedding(skel_pure_efa, skel_labels,skel_pure_means_efa, skel_means.index, title="EFA-space embedding of the skeletal data (EFA on skel data)", three_d=False)


'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches

'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.
'c' argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with 'x' & 'y'.  Please use a 2-D array with a single row if you really want to specify the same RGB or RGBA value for all points.


Adding the EFA projections to the original dataframes:

In [29]:
fullscene_df["efa1"] = pd.Series(fullscene_efa[:,0], index=fullscene_df.index)
fullscene_df["efa2"] = pd.Series(fullscene_efa[:,1], index=fullscene_df.index)
fullscene_df["efa3"] = pd.Series(fullscene_efa[:,2], index=fullscene_df.index)
skel_df["efa1"] = pd.Series(skel_efa[:,0], index=skel_df.index)
skel_df["efa2"] = pd.Series(skel_efa[:,1], index=skel_df.index)
skel_df["efa3"] = pd.Series(skel_efa[:,2], index=skel_df.index)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

In [30]:
skel_efa

array([[ 2.76811863, -4.30307959, -3.04140706],
       [13.21308183, -5.03815582, -2.11413087],
       [ 5.20458581, -5.36950514,  8.33845159],
       ...,
       [ 9.13070548,  2.58908914,  0.88625838],
       [ 7.7538076 , -0.6148631 ,  0.60718982],
       [ 9.26433542,  0.85983507,  1.88617936]])

Interestingly, even if the EFA factors are quite similar, the distances between same clips in fullscene vs skeletal data are higher in the EFA space compared to the PCA or LDA space:

In [31]:
distances_efa=pd.DataFrame(np.power(np.sum(np.power(skel_means_efa - fullscene_means_efa, 2), axis=1), 0.5), index=skel_means.index, columns=["distance_efa"])

distances = pd.concat([distances_pca, distances_lda, distances_efa], axis=1)
print("Mean distances:\n%s" % distances.mean(axis=0))
show_heatmap(distances, cmap="summer_r")


Mean distances:
distance_pca    2.292290
distance_lda    0.948686
distance_efa    3.428116
dtype: float64


,distance_pca,distance_lda,distance_efa
clipId,,,
01,4.65382,1.78732,6.67003
02,0.883654,0.350891,1.64351
03,1.73115,1.03464,1.98804
04,4.40108,2.42992,4.6131
05,1.2188,0.486269,2.23726
06,2.3967,0.679606,4.56893
07,2.65566,1.27701,4.53029
08,1.57035,0.72836,1.74438
09,1.54301,0.63264,2.6114


# Clustering



Based on the distance measurements, the LDA space seems to be the best to cluster our clips.
We can then attempt to cluster our 20 clips into 'groups' of similar clips (based on the latent constructs):

In [32]:
from sklearn.cluster import KMeans

# kMeans clustering after projecting our clips in the EFA-space
fullscene_clustering_data=fullscene_means_efa

nb_clusters=7

fullscene_kmeans_model = KMeans(n_clusters=nb_clusters, random_state=0).fit(fullscene_clustering_data)
fullscene_kmeans = fullscene_kmeans_model.predict(fullscene_clustering_data)

plot_embedding(fullscene_clustering_data,fullscene_means.index,clusters=fullscene_kmeans, three_d=True)

pd.DataFrame(fullscene_kmeans, index=fullscene_means.index, columns=["cluster #"]).sort_values(by="cluster #")



,cluster #
clipId,
02,0
17,0
16,0
09,0
04,1
06,1
05,2
08,2
01,3


We should be able to infer the semantics of the 3 first EFA factors.


We can then try to predict in which cluster the clips would end up, using only the ratings from the skeletal videos:

In [33]:
skel_efa

array([[ 2.76811863, -4.30307959, -3.04140706],
       [13.21308183, -5.03815582, -2.11413087],
       [ 5.20458581, -5.36950514,  8.33845159],
       ...,
       [ 9.13070548,  2.58908914,  0.88625838],
       [ 7.7538076 , -0.6148631 ,  0.60718982],
       [ 9.26433542,  0.85983507,  1.88617936]])

In [34]:
skel_kmeans= fullscene_kmeans_model.predict(skel_means_lda)

plot_embedding(fullscene_means_lda, skel_means.index, clusters=skel_kmeans, three_d=False)

diff=pd.DataFrame(fullscene_kmeans-skel_kmeans,index=skel_means.index)
print("%d skeleton clips out of %d (%.1f%%) are predicted to fall into the same cluster as their 'fullscene' counterpart." % (diff[diff==0].count(), skel_kmeans.size, diff[diff==0].count() * 100. / skel_kmeans.size))

clusters_kripp=pd.DataFrame([fullscene_kmeans, skel_kmeans, fullscene_kmeans==skel_kmeans,krippendorff_df[["alpha(fullscene)", "alpha(skel)"]].std(axis=1).astype(float), krippendorff_df[["alpha(fullscene)", "alpha(skel)"]].mean(axis=1).astype(float), krippendorff_df["alpha(fullscene)"], krippendorff_df["alpha(skel)"], ],index=["fullscene clusters", "skel clusters", "same", "kripp alpha std", "kripp alpha mean", "alpha(fullscene)", "alpha(skel)"],columns=skel_means.index).T.sort_values(by="kripp alpha mean")
clusters_kripp

ValueError: Incorrect number of features. Got 4 features, expected 3

Is there a correlation between 'same clusters' and Krippendorf agreement (ie, consistency of ratings for a given clip)? No...

In [35]:
print("Mean Krippendorf, same cluster: %f" % clusters_kripp[clusters_kripp["same"] == True]["kripp alpha mean"].mean())
print("Std Krippendorf, same cluster: %f" % clusters_kripp[clusters_kripp["same"] == True]["kripp alpha mean"].std())
print("Mean Krippendorf, diff cluster: %f" % clusters_kripp[clusters_kripp["same"] == False]["kripp alpha mean"].mean())
print("Std Krippendorf, diff cluster: %f" % clusters_kripp[clusters_kripp["same"] == False]["kripp alpha mean"].std())

NameError: name 'clusters_kripp' is not defined

# Classification



Training a multi-label classifier on our clips

In [36]:
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer



training_ground_truth = { '01': ['Aggressive'],
                         '02': ['Excited', 'Aggressive', 'Aimless'],
                         '03': ['Excited', 'Fun'],
                         '04': ['Cooperative'],
                         '05': ['Bored', 'Aimless'],
                         '06': ['Cooperative'],
                         '07': ['Dominant'],
                         '08': ['Bored', 'Fun'],
                         '09': ['Cooperative'],
                         '10': ['Cooperative', 'Dominant'],
                         '11': ['Cooperative', 'Dominant'],
                         '12': ['Aggressive', 'Aimless'],
                         '13': ['Excited', 'Aggressive', 'Aimless'],
                         '14': ['Aggressive'],
                         '15': ['Dominant'],
                         '16': ['Cooperative', 'Dominant'],
                         '17': ['Excited', 'Aggressive'],
                         '18': ['Aggressive', 'Dominant'],
                         '19': ['Dominant'],
                         '20': ['Excited']}

mlb = MultiLabelBinarizer()
mlb.fit(training_ground_truth.values())

def datasets(training=df, testing=None, cols=allQuestions, test_size=0.2, use_clip_id_as_label=False, random_labels=False):
    """Returns a training dataset and training labels, and a testing dataset and testing labels.
    
    If testing is None, it randomly splits the training dataframe (at test_size).
    """


    if testing is None:
        
        if use_clip_id_as_label:
            labels = list(training["clipId"].map(int))
        else:
            labels = []
            for id in training["clipId"]:
                labels.append(training_ground_truth[id])

        data = training[cols].values

        training_data, testing_data, training_labels, testing_labels = train_test_split(data, labels, test_size=test_size, random_state=int(time.time()))

        if not use_clip_id_as_label:
            
            training_labels, testing_labels = mlb.transform(training_labels), mlb.transform(testing_labels)
            
            if random_labels:
                for labels in training_labels:
                    np.random.shuffle(labels)                 
                np.random.shuffle(training_labels)             
            

        return training_data, testing_data, training_labels, testing_labels
    
    else:
        
        if use_clip_id_as_label:
            training_labels = list(training["clipId"].map(int))
            testing_labels = list(testing["clipId"].map(int))
        else:
            labels = []
            for id in training["clipId"]:
                labels.append(training_ground_truth[id])

            training_labels = mlb.transform(labels)

            labels = []
            for id in testing["clipId"]:
                labels.append(training_ground_truth[id])

            testing_labels = mlb.transform(labels)

            if random_labels:
                if random_labels:
                    for labels in training_labels:
                        np.random.shuffle(labels)                 
                    np.random.shuffle(training_labels) 

        
        training_data = training[cols].values
        testing_data = testing[cols].values

        return training_data, testing_data, training_labels, testing_labels

In [37]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

def train(training_data, training_labels, layers_nb=3, layer_size=20):
    
    #clf = RandomForestClassifier()
    clf = KNeighborsClassifier(n_neighbors=3)
    #clf = ExtraTreeClassifier(random_state=0)

    layers = (layer_size, ) * layers_nb
    #print("Training a MLP classifier, layers: %s..." % str(layers))

    # => naively using a OneVsRestClassifier does not improve the classification results, on the contrary
    #clf = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes=layers, activation='relu', max_iter=1000, solver="lbfgs"))
    
    #clf = MLPClassifier(hidden_layer_sizes=layers, activation='relu', max_iter=1000, solver="lbfgs")
    #clf = OneVsRestClassifier(SVC(kernel='rbf'))
    
    clf.fit(training_data, training_labels)
    
    return clf

def predict(clf, testing_data, inverse_transform_labels=True):
    p = clf.predict(testing_data)
    if inverse_transform_labels:
        return mlb.inverse_transform(p) 
    else:
        return p

In [38]:
import sklearn
import sklearn.metrics as metrics

def run_classification(training, 
                       testing=None, 
                       cols=allQuestions, 
                       use_clip_id_as_label=False, 
                       random_labels=False,
                       layers_nb=3,
                       layer_size=20,
                       iterations=50):
    """
    Metrics for multi-label classification coming form Sorower, Mohammad S. "A literature survey on algorithms for multi-label learning." Oregon State University, Corvallis (2010).
    """
    
    results = {"Accuracy": [],
               #"Jaccard similarity" : [], # intersection over union
              "Precision": [],
               "Recall": [],
              "F1-measure": []}              
              #"hamming_loss": []} # not super useful, because as we have 'only' 7 labels, the hamming distance is never huge (max 7, and usually smaller), which make it a not-very-sensitive measure
    labels_f1 = []
    

    for x in range(iterations):
               
        training_data, testing_data, training_labels, testing_labels = datasets(training=training, testing=testing, cols=cols, use_clip_id_as_label=use_clip_id_as_label, random_labels=random_labels)
        
        if x == 0:
            print("Shape of training data: %s" % str(training_data.shape))
            print("Shape of testing data: %s" % str(testing_data.shape))

        #print("Fold %d/%d" % (x+1, iterations))
        
        clf = train(training_data, training_labels, layers_nb=layers_nb, layer_size=layer_size)

        #mean_score_exact += clf.score(testing_data, testing_labels)

        pred_labels = predict(clf, testing_data, inverse_transform_labels = not use_clip_id_as_label)


        
        at_least_one = 0
        at_least_one_no_incorrect = 0
        
        if not use_clip_id_as_label:
            
            nb_classes = len(mlb.classes_)
            
            labels_f1.append(dict(zip(mlb.classes_, metrics.f1_score(testing_labels, mlb.transform(pred_labels), average=None))))
            
            results["Accuracy"].append(metrics.accuracy_score(testing_labels, mlb.transform(pred_labels)))
            #results["Jaccard similarity"].append(metrics.jaccard_similarity_score(testing_labels, mlb.transform(pred_labels)))
            results["Precision"].append(metrics.precision_score(testing_labels, mlb.transform(pred_labels), average='weighted'))    
            results["Recall"].append(metrics.recall_score(testing_labels, mlb.transform(pred_labels), average='weighted'))    
            results["F1-measure"].append(metrics.f1_score(testing_labels, mlb.transform(pred_labels), average='weighted'))    
            #results["hamming_loss"].append(metrics.hamming_loss(testing_labels, mlb.transform(pred_labels)))    
            
            
            testing_labels = mlb.inverse_transform(testing_labels)
            
            exact = 0
            accuracy = 0
            precision = 0
            recall = 0
            f1_measure = 0
            
            for actual, pred in zip(testing_labels, pred_labels):
                
                pred = set(pred)
                actual = set(actual)
                
                if len(pred) == 0: continue
                    
                if pred == actual:
                    #print("%s <-> %s" % (actual, pred))
                    exact += 1
                    
                intersection = pred.intersection(actual)
                union = pred.union(actual)

                #accuracy += float(len(intersection)) / len(union)
                #precision += float(len(intersection)) / len(pred)
                #recall += float(len(intersection)) / len(actual)
                #f1_measure += 2 * float(len(intersection)) / (len(pred) + len(actual))
                
            
            #results["exact"].append(float(exact) / len(testing_labels))
            #results["accuracy"].append(accuracy / len(testing_labels))
            #results["precision"].append(precision / len(testing_labels))
            #results["recall"].append(recall / len(testing_labels))
            #results["f1_measure"].append(f1_measure / len(testing_labels))
            
            
            
        else: # use_clip_id_as_label = True
            # does not make much sense as at_least_one & at_least_one_no_incorrect are the same as 'exact'
            pass

    return pd.DataFrame(results), pd.DataFrame(labels_f1)

### Multi-label classification

MLP hyperparameters optimisation using a grid search

In [39]:
res = {}
for nb in range(1,6):
    for size in range(2,21,2): 
        results_fullscene,labels_precision_fullscene = run_classification(fullscene_df, iterations=30,layers_nb=nb, layer_size=size)
        res["(%d x %d)" % (nb, size)] = results_fullscene
        #print(results_fullscene)

grid_search=pd.concat(res, axis=1)
grid_search.describe()

Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Shape of t

(1 x 10)                                                (1 x 12)  \
           Accuracy     Precision        Recall    F1-measure      Accuracy   
count  3.000000e+01  3.000000e+01  3.000000e+01  3.000000e+01  3.000000e+01   
mean   1.375000e-01  4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   
std    2.823006e-17  1.129203e-16  1.129203e-16  1.129203e-16  2.823006e-17   
min    1.375000e-01  4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   
25%    1.375000e-01  4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   
50%    1.375000e-01  4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   
75%    1.375000e-01  4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   
max    1.375000e-01  4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   

                                                     (1 x 14)             \
          Precision        Recall    F1-measure      Accuracy  Precision   
count  3.000000e+01  3.000000e+01  3.000000e+01  3.000000e+01  30.000000   
mean   4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   0.483694   
std    1.129203e-16  1.129203e-16  1.129203e-16  2.823006e-17   0.001268   
min    4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   0.479954   
25%    4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   0.484110   
50%    4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   0.484110   
75%    4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   0.484110   
max    4.799535e-01  3.257576e-01  3.753429e-01  1.375000e-01   0.484110   

         ...           (5 x 4)                     (5 x 6)                \
         ...            Recall    F1-measure      Accuracy     Precision   
count    ...      3.000000e+01  3.000000e+01  3.000000e+01  3.000000e+01   
mean     ...      3.730159e-01  3.991428e-01  1.625000e-01  4.440559e-01   
std      ...      1.129203e-16  1.129203e-16  5.646013e-17  1.129203e-16   
min      ...      3.730159e-01  3.991428e-01  1.625000e-01  4.440559e-01   
25%      ...      3.730159e-01  3.991428e-01  1.625000e-01  4.440559e-01   
50%      ...      3.730159e-01  3.991428e-01  1.625000e-01  4.440559e-01   
75%      ...      3.730159e-01  3.991428e-01  1.625000e-01  4.440559e-01   
max      ...      3.730159e-01  3.991428e-01  1.625000e-01  4.440559e-01   

                                     (5 x 8)                                   
             Recall    F1-measure   Accuracy  Precision     Recall F1-measure  
count  3.000000e+01  3.000000e+01  30.000000  30.000000  30.000000  30.000000  
mean   3.730159e-01  3.991428e-01   0.160000   0.438007   0.369841   0.394012  
std    1.129203e-16  1.129203e-16   0.005085   0.012305   0.006458   0.010436  
min    3.730159e-01  3.991428e-01   0.150000   0.413811   0.357143   0.373490  
25%    3.730159e-01  3.991428e-01   0.162500   0.444056   0.373016   0.399143  
50%    3.730159e-01  3.991428e-01   0.162500   0.444056   0.373016   0.399143  
75%    3.730159e-01  3.991428e-01   0.162500   0.444056   0.373016   0.399143  
max    3.730159e-01  3.991428e-01   0.162500   0.444056   0.373016   0.399143  

[8 rows x 200 columns]

In [40]:
accuracy_over_grid = grid_search.swaplevel(axis=1)["F1-measure"].describe().transpose()
accuracy_over_grid["mean"].plot.bar(yerr=accuracy_over_grid["std"])


/home/s-lemaignan/.local/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [41]:
results_fullscene,labels_precision_fullscene = run_classification(fullscene_df, iterations=10,layers_nb=1, layer_size=6)
results_fullscene.describe()

Shape of training data: (316, 30)
Shape of testing data: (80, 30)


,Accuracy,Precision,Recall,F1-measure
count,1.000000e+01,1.000000e+01,10.000000,10.000000
mean,1.625000e-01,3.742687e-01,0.286885,0.314139
std,2.925695e-17,5.851389e-17,0.000000,0.000000
min,1.625000e-01,3.742687e-01,0.286885,0.314139
25%,1.625000e-01,3.742687e-01,0.286885,0.314139
50%,1.625000e-01,3.742687e-01,0.286885,0.314139
75%,1.625000e-01,3.742687e-01,0.286885,0.314139
max,1.625000e-01,3.742687e-01,0.286885,0.314139


In [42]:
nb_iterations = 20

print("Fullscene, 80%/20%...")
results_fullscene,labels_f1_fullscene = run_classification(fullscene_df, iterations=nb_iterations)
print("Fullscene, 80%/20%, EFA space...")
results_fullscene_efa,labels_f1_fullscene_efa = run_classification(fullscene_df, cols=["efa1", "efa2", "efa3"], iterations=nb_iterations)
print("Fullscene, chance level...")
results_fullscene_chance,labels_f1_fullscene_chance = run_classification(fullscene_df, random_labels=True, iterations=nb_iterations)
print("Fullscene, 80%/20%, sanity check [input cols=['age']]...")
results_fullscene_age,labels_f1_fullscene_age = run_classification(fullscene_df, cols=["age"], iterations=nb_iterations)

print("Fullscene vs skeletons...")
results_fullscene_skel,labels_f1_skel = run_classification(fullscene_df, testing=skel_df, iterations=nb_iterations)
print("Fullscene vs skeletons, EFA space...")
results_fullscene_skel_efa,labels_f1_skel_efa = run_classification(fullscene_df, testing=skel_df, cols=["efa1", "efa2", "efa3"], iterations=nb_iterations)
print("Fullscene vs skeletons, chance level...")
results_fullscene_skel_chance,labels_f1_skel_chance = run_classification(fullscene_df, testing=skel_df, random_labels=True, iterations=nb_iterations)

collated_results = pd.DataFrame({"Full-scene": results_fullscene.mean(),
                                 "Full-scene, EFA space": results_fullscene_efa.mean(),
                                 "Full-scene, chance": results_fullscene_chance.mean(),
                                 #"Full-scene-80-20-sanity-check": results_fullscene_age.mean(),
                                 "Movement-alone": results_fullscene_skel.mean(),
                                 "Movement-alone, EFA space": results_fullscene_skel_efa.mean(),
                                 "Movement-alone, chance": results_fullscene_skel_chance.mean()})
labels_f1 = pd.concat({"Full-scene": labels_f1_fullscene,
                              "Full-scene, EFA space": labels_f1_fullscene_efa,
                              "Full-scene, chance": labels_f1_fullscene_chance,
                              #"fullscene-80-20-sanity-check": labels_f1_fullscene_age,
                              "Movement-alone": labels_f1_skel,
                              "Movement-alone, EFA": labels_f1_skel_efa,
                              "Movement-alone, chance": labels_f1_skel_chance}, axis=1)


Fullscene, 80%/20%...
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Fullscene, 80%/20%, EFA space...
Shape of training data: (316, 3)
Shape of testing data: (80, 3)
Fullscene, chance level...
Shape of training data: (316, 30)
Shape of testing data: (80, 30)
Fullscene, 80%/20%, sanity check [input cols=['age']]...
Shape of training data: (316, 1)
Shape of testing data: (80, 1)
Fullscene vs skeletons...
Shape of training data: (396, 30)
Shape of testing data: (400, 30)
Fullscene vs skeletons, EFA space...
Shape of training data: (396, 3)
Shape of testing data: (400, 3)
Fullscene vs skeletons, chance level...
Shape of training data: (396, 30)
Shape of testing data: (400, 30)


In [50]:
print((collated_results*100).round(1).transpose().to_latex(bold_rows=True))

\begin{tabular}{lrrrr}
\toprule
{} &  Accuracy &  Precision &  Recall &  F1-measure \\
\midrule
\textbf{Full-scene               } &      17.0 &       46.2 &    33.6 &        37.4 \\
\textbf{Full-scene, EFA space    } &      12.5 &       42.2 &    28.5 &        32.6 \\
\textbf{Full-scene, chance       } &       3.1 &       24.3 &    14.0 &        17.0 \\
\textbf{Movement-alone           } &      15.8 &       41.6 &    32.7 &        36.3 \\
\textbf{Movement-alone, EFA space} &      11.7 &       35.1 &    27.0 &        30.3 \\
\textbf{Movement-alone, chance   } &       4.4 &       28.2 &    14.6 &        18.3 \\
\bottomrule
\end{tabular}



In [75]:
labels_fullscene_skel=labels_f1.describe().transpose()["mean"].unstack()

#labels_fullscene_skel["mean"].plot.bar(yerr=labels_fullscene_skel["std"])
#print((labels_fullscene_skel*100).round(1).to_latex(bold_rows=True))
labels_fullscene_skel.transpose()[["Full-scene", "Movement-alone"]].plot.bar()

## Classification over the k-means clusters

In [ ]:
clf_clusters = MLPClassifier(hidden_layer_sizes=(20,20,20), activation='relu', max_iter=1000, solver="lbfgs")

# trying with skel, skel_pca, skel_lda do not lead to any clear improvements
training_set = fullscene_lda

training_labels = []

for id in fullscene_df["clipId"]:
    training_labels.append(fullscene_kmeans[int(id)-1])

testing_labels = []

for id in range(20):
    testing_labels.append(fullscene_kmeans[int(id)-1])

testing_set = skel_means_lda


clf_clusters.fit(training_set, training_labels)

print("Full scene clusters:    " + str(fullscene_kmeans))
print("Predicted clusters skel:" + str(clf_clusters.predict(testing_set)))

clf_clusters.score(testing_set, testing_labels)

Using a SVM classifier, we can try to improve our predictions:

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='rbf')

# trying with skel, skel_pca, skel_lda do not lead to any clear improvements
training_set = fullscene_means_lda
training_labels = fullscene_kmeans

testing_set = skel_means_lda
# Critically, we are testing with the *fullscene_kmeans* as we want to check whether 
# we predict the same clusters as with the fullscene stimuli.
testing_labels = fullscene_kmeans

clf.fit(training_set, training_labels)

In [ ]:
for p, l in zip(clf.predict(testing_set), testing_labels):
    print("%s (should be %s)" % (p,l))
print("SVM: %.1f%% successful prediction out of %d tested clips" % (clf.score(testing_set, testing_labels) * 100, len(testing_labels)))

In [ ]:
from sklearn.metrics import confusion_matrix

cnf_matrix = confusion_matrix(testing_labels, clf.predict(testing_set))
plot_confusion_matrix(cnf_matrix, classes=pd.unique(testing_labels))